# Getting Started

This notebook is to help new users get started with using the `pysplot` python package for spatiotemporal plotting.
We will run through examples of how to retrieve spatial and science data and combine it into an object that will be suitable for `pysplot` plotting.

In [ ]:
## install if not present
### !pip install pyspedas

In [10]:
from sunpy.coordinates import GeocentricSolarEcliptic
import astropy.coordinates
import astropy.units as u
import pyspedas

from pysplot.io.data import SpatialData, ScienceData, SpatialTimeData

### Step 1: Create SpatialData and ScienceData object

Here we will step through an example of splotting MMS data for one month.

SpatialData is an object containing information on the location of the spacecraft.
SpatialData expects data to be fed in with a dictionary where 'x' is a list of epochs in datetime format and 'y' contains the values.
If you use `pyspedas` to retrieve the data, the data will output in the correct format if the keyword `notplot=True`. Otherwise, you could read in the data however you like and make sure the input dictionary contains 'x' and 'y' in the same format.

Splot is designed work with `sunpy` and `astropy`, especially the coordinate frames and units.

In [2]:
mms_mec_data = pyspedas.projects.mms.mms_load_mec(trange=['2020-01-01', '2020-01-05'], notplot=True)
spatial_data = mms_mec_data['mms1_mec_r_geo']
spatial_data

16-Jan-25 19:51:26: Loading pydata/mms1/mec/srvy/l2/epht89q/2020/01/mms1_mec_srvy_l2_epht89q_20200101_v2.2.0.cdf
16-Jan-25 19:51:26: Loading pydata/mms1/mec/srvy/l2/epht89q/2020/01/mms1_mec_srvy_l2_epht89q_20200102_v2.2.0.cdf
16-Jan-25 19:51:26: Loading pydata/mms1/mec/srvy/l2/epht89q/2020/01/mms1_mec_srvy_l2_epht89q_20200103_v2.2.0.cdf
16-Jan-25 19:51:26: Loading pydata/mms1/mec/srvy/l2/epht89q/2020/01/mms1_mec_srvy_l2_epht89q_20200104_v2.2.0.cdf
16-Jan-25 19:51:26: Floating point data values for variable mms1_mec_loss_cone_angle_s are all fillval (-1.000000e+31)
16-Jan-25 19:51:26: Floating point data values for variable mms1_mec_loss_cone_angle_n are all fillval (-1.000000e+31)
16-Jan-25 19:51:26: Floating point data values for variable mms1_mec_pfs_geod_latlon are all fillval (-1.000000e+31)
16-Jan-25 19:51:26: Floating point data values for variable mms1_mec_pfn_geod_latlon are all fillval (-1.000000e+31)
16-Jan-25 19:51:26: Floating point data values for variable mms1_mec_pfs_gsm

{'x': array(['2020-01-01T00:00:00.000000000', '2020-01-01T00:00:30.000000000',
        '2020-01-01T00:01:00.000000000', ...,
        '2020-01-04T23:58:30.000000000', '2020-01-04T23:59:00.000000000',
        '2020-01-04T23:59:30.000000000'], dtype='datetime64[ns]'),
 'y': array([[-103022.14797985,  -22630.7481424 ,    6293.65967293],
        [-103114.89881914,  -22439.35411279,    6286.36814742],
        [-103207.2737431 ,  -22247.65570303,    6279.07470234],
        ...,
        [-145969.04484364,  -55461.11615224,   -4884.96048988],
        [-146106.8419048 ,  -55165.44524281,   -4892.73738136],
        [-146244.03003486,  -54869.43150151,   -4900.51379811]])}

In [3]:

spatial_data_obj = SpatialData(spatial_data, coord=astropy.coordinates.ITRS, units=u.km, spatial_columns_names=['x_r_geo', 'y_r_geo', 'z_r_geo'], coord_kwargs={'representation_type':'cartesian'})

In [4]:
print(spatial_data_obj.data.head(10))
print('coordinates', spatial_data_obj.coord)
print('units', spatial_data_obj.units)

                           x_r_geo       y_r_geo      z_r_geo
2020-01-01 00:00:00 -103022.147980 -22630.748142  6293.659673
2020-01-01 00:00:30 -103114.898819 -22439.354113  6286.368147
2020-01-01 00:01:00 -103207.273743 -22247.655703  6279.074702
2020-01-01 00:01:30 -103299.271825 -22055.653658  6271.779326
2020-01-01 00:02:00 -103390.892258 -21863.348797  6264.482051
2020-01-01 00:02:30 -103482.134260 -21670.741971  6257.182879
2020-01-01 00:03:00 -103572.996908 -21477.833917  6249.881817
2020-01-01 00:03:30 -103663.479339 -21284.625433  6242.578875
2020-01-01 00:04:00 -103753.580509 -21091.117304  6235.274052
2020-01-01 00:04:30 -103843.299833 -20897.310501  6227.967312
coordinates <class 'astropy.coordinates.builtin_frames.itrs.ITRS'>
units km


## Science Data

Most of the time, you also want to consider some scientific data to go along with the spatial data for plotting. Here's how you would initialize the `ScienceData` object, similar to `SpatialData`.

We'll pull magnetic field from the MMS FGM instrument for the same time period and initialize our `ScienceData` object.

In [5]:
mms_fgm_data = pyspedas.projects.mms.mms_load_fgm(trange=['2020-01-01', '2020-01-05'], notplot=True)
science_data = mms_fgm_data['mms1_fgm_b_gse_srvy_l2']
science_data


16-Jan-25 19:51:31: Loading pydata/mms1/fgm/srvy/l2/2020/01/mms1_fgm_srvy_l2_20200101_v5.224.0.cdf
16-Jan-25 19:51:31: Loading pydata/mms1/fgm/srvy/l2/2020/01/mms1_fgm_srvy_l2_20200102_v5.224.0.cdf
16-Jan-25 19:51:31: Loading pydata/mms1/fgm/srvy/l2/2020/01/mms1_fgm_srvy_l2_20200103_v5.224.0.cdf
16-Jan-25 19:51:31: Loading pydata/mms1/fgm/srvy/l2/2020/01/mms1_fgm_srvy_l2_20200104_v5.226.0.cdf


{'x': array(['2020-01-01T00:00:02.648887922', '2020-01-01T00:00:02.711388827',
        '2020-01-01T00:00:02.773889732', ...,
        '2020-01-05T00:00:14.645025203', '2020-01-05T00:00:14.770026980',
        '2020-01-05T00:00:14.895028757'], dtype='datetime64[ns]'),
 'y': array([[ -2.575452 ,   4.1985846, -22.964613 ,  23.486898 ],
        [ -2.6951935,   2.5182579, -22.603752 ,  22.902735 ],
        [ -3.485547 ,   0.6691314, -22.953402 ,  23.226177 ],
        ...,
        [  2.6056378,  -8.73475  ,  -0.8176866,   9.151711 ],
        [  2.618569 ,  -8.728434 ,  -0.8134932,   9.149002 ],
        [  2.6257954,  -8.727181 ,  -0.8250758,   9.150915 ]],
       dtype=float32)}

In [6]:
science_data_obj = ScienceData(science_data, units=1e-9*u.T, science_columns_names=['Bx_GSE', 'By_GSE', 'Bz_GSE'])

In [7]:
print(science_data_obj.data.head(10))
print('units', science_data_obj.units)

                                 Bx_GSE    By_GSE     Bz_GSE
2020-01-01 00:00:02.648887922 -2.575452  4.198585 -22.964613
2020-01-01 00:00:02.711388827 -2.695194  2.518258 -22.603752
2020-01-01 00:00:02.773889732 -3.485547  0.669131 -22.953402
2020-01-01 00:00:02.836390638 -3.912055 -1.209539 -24.060274
2020-01-01 00:00:02.898891543 -3.625044 -1.266421 -24.118010
2020-01-01 00:00:02.961392449 -3.110532 -2.144957 -25.306442
2020-01-01 00:00:03.023893354 -4.032660 -2.470277 -27.366508
2020-01-01 00:00:03.086394260 -4.361229 -1.059933 -27.157602
2020-01-01 00:00:03.148895165 -3.582466  1.038460 -27.104290
2020-01-01 00:00:03.211396070 -2.496563  1.200684 -26.587170
units 1e-09 T


## Create the combined data

Now that you have both `SpatialData` and `ScienceData` initialized, you can combine them into an object for splotting (`SpatialTimeData`).

The spatial data was in GEO coordinates; however the science data was pulled in GSE coordinates. To rectify that, we can convert the spatial data to GSE to match the science data by with the `desired_coord` argument.


In [8]:
splot_data_obj = SpatialTimeData(spatial_data_obj, science=science_data_obj, desired_units=1e-9*u.T, desired_coord=GeocentricSolarEcliptic, desired_coord_kwargs={'representation_type':'cartesian'})

In [9]:

print(splot_data_obj.data.head(10))
print('splot spatial columns', splot_data_obj.spatial_columns)
print('splot coord', splot_data_obj.coord)
print('splot science columns', splot_data_obj.science_columns)
print('splot units', splot_data_obj.units)


                                     lon        lat       distance    Bx_GSE  \
2020-01-01 00:00:02.648887922  14.910629  24.924501  105666.208298 -2.575452   
2020-01-01 00:00:02.711388827  14.910658  24.924483  105666.320388 -2.695193   
2020-01-01 00:00:02.773889732  14.910687  24.924466  105666.432477 -3.485547   
2020-01-01 00:00:02.836390638  14.910716  24.924448  105666.544567 -3.912054   
2020-01-01 00:00:02.898891543  14.910745  24.924431  105666.656656 -3.625044   
2020-01-01 00:00:02.961392449  14.910774  24.924413  105666.768746 -3.110531   
2020-01-01 00:00:03.023893354  14.910803  24.924396  105666.880835 -4.032660   
2020-01-01 00:00:03.086394260  14.910832  24.924378  105666.992925 -4.361229   
2020-01-01 00:00:03.148895165  14.910861  24.924361  105667.105014 -3.582466   
2020-01-01 00:00:03.211396070  14.910890  24.924344  105667.217104 -2.496563   

                                 By_GSE     Bz_GSE  
2020-01-01 00:00:02.648887922  4.198585 -22.964611  
2020-01-01 00

## TBD: Example of spatiotemporal plotting using SpatialTimeData object

#### Transform from one coordinate frame to another

Since we used sunpy coordinate system to define the splot data spatial frame, we can use the underlying functionality of sunpy to transform the spatial coordinate from one frame to another. For example, we are currently in GeocentricSolarEcliptic with coordinate units in km, but we can transform the splot merged data to be presented in Geocentric Solar Magnetospheric frame with units of AU instead. Note that sunpy coordinate transformations can be very timely if the observing time is required to calculate the location of the Sun and Earth.

In [17]:
## TODO: make this work
#splot_data_obj.transform_coord(Geomagnetic, desired_units=u.AU)
#splot_data_obj.data